In [ ]:
import pandas as pd
import numpy as np
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import plotly.graph_objects as go

# Codes for importing the excel or csv dataset
df1 = pd.read_excel(r"TYPE THE DIRECTORY PATH OF YOUR EXCEL FILE") # Seismic Velocity Data Input File #
df2 = pd.read_excel(r"TYPE THE DIRECTORY PATH OF YOUR EXCEL FILE") # Well Data Input File #

# Application building codes
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
colorscales = px.colors.named_colorscales()

# Lists #
twt_list = [i for i in range (500, 10500, 500)]

app = JupyterDash (__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div ([
    html.H2 ("Velocitemperature v1.0", style={"font-weight":"bold", "font-size":"22px"}),
    html.P ("2D Temperature profiles are displayed below."
            " You can alter the profiles by entering different values for constants A and B.", style={"font-weight":"bold",
                                                                                                   "font-size":"16px"}),
    html.Div ([
        html.Label ("Constant A", style={"display":"inline-block", "margin-left":"2em", 
                                         "font-size":"16px", "font-weight":"bold"}),
        html.Label ("Constant B", style={"display":"inline-block", "margin-left":"8.5em", 
                                         "font-size":"16px", "font-weight":"bold"}),
        html.Label ("Mode", style={"display":"inline-block", "margin-left":"15em", 
                                         "font-size":"16px", "font-weight":"bold"}),
        html.Label ("Coordinate Filter", style={"display":"inline-block", "margin-left":"18.5em", 
                                         "font-size":"16px", "font-weight":"bold"})
    ], style={"display":"flex", "margin-top":"2.5em"}),
    
    html.Div ([
        html.Div ([
            dcc.Input (id="constant-a", type="number", value=0.08, style={"textAlign":"center","width":"100px", 
                                                                          "margin-left":"2em", "display":"inline-block"}),
            dcc.Input (id="constant-b", type="number", value=0.003, style={"textAlign":"center","width":"100px", 
                                                                          "margin-left":"8em", "display":"inline-block"})
        ]),
        
        html.Div ([
            dcc.RadioItems (id="modes", options=[{"label":"Well Calibration", "value":"Well Calibration"}, 
                            {"label":"Full View Mode", "value":"Full View Mode"}], value="Full View Mode",
                            labelStyle={"display":"inline-block", "margin-left":"5em"}),
            dcc.Dropdown (id="coordinates",
                                    options=[{"label": i, "value": i} for i in df1["X"]],
                                    style={"display":"inline-block", "width":"250px", "margin-left":"5em"},
                                    placeholder="Please select a coordinate", 
                                    clearable=False, disabled=False, multi=False)
        ], style={"display":"flex", "flex-direction":"row"})    
    ], style={"display":"flex", "flex-direction":"row", "margin-top":"0.5em"}),
    
    html.Br (),
    html.Button ("Export Data", id="export", style={"textAlign":"center", "backgroundColor":"#3283FE", 
                                                            "color":"#E2E2E2", "display":"inline-block", 
                                                            "margin-left":"90em"}, n_clicks=0),
    html.Hr (),
    html.P ("2D CHART VIEW", style={"font-size":"18px", "font-weight":"bold"}),
    html.Div ([
        html.Div ([
            dcc.Graph (id="time_graph", style={"display":"inline-block", "width":"50%"}),
            dcc.Graph (id="depth_graph", style={"display":"inline-block", "width":"50%"})
        ])
    ]),
    
    html.P ("2D MAP VIEW", style={"font-size":"18px", "font-weight":"bold"}),
    html.Div ([
        html.Label ("Map Type", style={"display":"inline-block", "margin-left":"2em", 
                                         "font-size":"16px", "font-weight":"bold"}),
        html.Label ("Timeslice (ms.)", style={"display":"inline-block", "margin-left":"18.5em", 
                                         "font-size":"16px", "font-weight":"bold"}),
        html.Label ("Colourscale", style={"display":"inline-block", "margin-left":"13.2em", 
                                         "font-size":"16px", "font-weight":"bold"}),
    ], style={"display":"flex", "flex-direction":"row"}),
    
    html.Div ([
        dcc.RadioItems (id="maptypes", options=[{"label":"Heat Map", "value":"Heat Map"}, 
                            {"label":"Contour Map", "value":"Contour Map"}], value="Contour Map",
                            labelStyle={"display":"inline-block", "margin-left":"2em"}),
        dcc.Dropdown (id="depths",
                                    options=[{"label": i, "value": i} for i in twt_list],
                                    style={"display":"inline-block", "width":"250px", "margin-left":"5em"},
                                    value = 500, 
                                    clearable=False, disabled=False, multi=False),
        dcc.Dropdown (id="colourscale",
                                    options=[{"label": i, "value": i} for i in colorscales],
                                    style={"display":"inline-block", "width":"250px", "margin-left":"5em"},
                                    value="icefire", 
                                    clearable=False, disabled=False, multi=False),
    ], style={"display":"flex", "flex-direction":"row"}),
    
    html.Div ([
        dcc.Graph (id="2dmap")
    ]),
    
    html.Hr (),
    html.P ("3D VIEW", style={"font-size":"18px", "font-weight":"bold"}),
    dcc.Graph (id="3dpointdata"),
    html.Br (),
    dcc.Graph (id="3dview")
])

@app.callback (
     [Output("time_graph", "figure"),
      Output("depth_graph", "figure"),
      Output ("3dpointdata", "figure"),
      Output ("3dview", "figure")],
     [Input ("constant-a", "value"),
      Input ("constant-b", "value"),
      Input ("export", "n_clicks"),
      Input ("modes", "value"),
      Input ("coordinates", "value"),
      Input ("colourscale", "value")]
)

def figure_update (a, b, export, mode, drop, colours):
    # Codes for the calculation of Static Temperatures
    df1["NV"] = (df1["V"]-df1["V"].min())/(df1["V"].max()-df1["V"].min())    # Calculates the normalised values
    df1["ST"] = (df1["NV"]+a)/b                          # Calculates the static temperatures in Celcius degrees 
    
    # Codes for figure generation
    fig1 = px.scatter(df1, x="ST", y="T", color=df1["ST"], hover_data=["X", "Y"],
                     title="TWT vs. Temperature", labels={"ST" : "Temperatures (C)", "T" : "TWT (ms.)"})
    fig1.update_yaxes (autorange="reversed")
    fig1.update_layout (font_family="Arial")
    
    fig2 = go.Figure ()
    if mode == "Full View Mode":
        fig2 = px.scatter(df1, x="ST", y="TVD_m", color=df1["ST"], hover_data=["X", "Y"],
                         title="TVD vs. Temperature", labels={"ST" : "Temperatures (C)", "TVD_m" : "TVD (m.)"})
    elif mode == "Well Calibration":
        fig2.add_trace(go.Scatter (x=df1.loc[df1["X"] == drop, "ST"], y=df1.loc[df1["X"] == drop, "TVD_m"],
                                           mode="markers", name="Temperatures (C)", 
                                           marker_color=df1.loc[df1["X"] == drop, "ST"]))
        fig2.add_trace(go.Scatter(x=df2['T'], y=df2['TVD_m'], mode='markers', name='Well Data', 
                                  marker_symbol="diamond", marker_size=9, marker_color="black"))
           
    fig2.update_yaxes (autorange="reversed")
    fig2.update_layout (font_family="Arial")
    
    # 3D Scatter as Point Data #
    fig3 = go.Figure(data=[go.Scatter3d(x=df1["X"], y=df1["Y"], z=df1["ST"], 
                                        mode='markers', 
                                        marker=dict(size=6, color=df1["ST"], 
                                                    colorscale=colours, colorbar=dict(title="Temperatures (C)"))
    )])
    
    fig3.update_layout (title="Temperatures as Point Data", width=1200, height=500)
    
    # 3D Cube #
    fig4 = go.Figure(data=[
    go.Mesh3d(
        # 8 vertices of a cube
        x=[df1['X'].min (), df1['X'].min (), df1['X'].max (), df1['X'].max (), 
           df1['X'].min (), df1['X'].min (), df1['X'].max (), df1['X'].max ()],
        y=[df1['Y'].min (), df1['Y'].max (), df1['Y'].max (), df1['Y'].min (), df1['Y'].min (), 
           df1['Y'].max (), df1['Y'].max (), df1['Y'].min ()],
        z=[df1['ST'].min (), df1['ST'].min (), df1['ST'].min (), df1['ST'].min (), 
           df1['ST'].max (), df1['ST'].max (), df1['ST'].max (), df1['ST'].max ()],
        colorbar_title='Temperatures (C)',
        colorscale=colours,
        # Intensity of each vertex, which will be interpolated and color-coded
        intensity = np.linspace(df1['ST'].max (), df1['ST'].min (), 10, endpoint=True), 
        # i, j and k give the vertices of triangles
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
        showscale=True)
    ])
    fig4.update_layout (title="Temperature Cube", width=1200, height=500)
    
    if export >= 1:
        df1.to_excel ("Velocity-Temperature Conversion.xlsx")
    
    return fig1, fig2, fig3, fig4

@app.callback (
      Output("2dmap", "figure"),
     [Input ("maptypes", "value"),
      Input ("depths", "value"),
      Input ("colourscale", "value")]
)

def map_update (maptype, timeslice, colourscale):
    cc = df1.loc[df1["T"] == timeslice, "X"]
    dd = df1.loc[df1["T"] == timeslice, "Y"]
    ee = df1.loc[df1["T"] == timeslice, "ST"]
      
    if maptype == "Heat Map":
          map_fig = go.Figure (data=go.Heatmap (x=cc, y=dd, z=ee, connectgaps=True, zsmooth="best",
                                      colorbar=dict(
                                      title="Temperatures (C)",
                                      titleside="right",
                                      titlefont=dict(size=14, family="Arial, sans-serif")),
                                      colorscale=colourscale))
    elif maptype == "Contour Map":
          map_fig = go.Figure (data=go.Contour (x=cc, y=dd, z=ee, connectgaps=True,
                                      colorbar=dict(
                                      title="Temperatures (C)",
                                      titleside="right",
                                      titlefont=dict(size=14, family="Arial, sans-serif")),
                                      contours_coloring="heatmap", line_smoothing=1.2,
                                      colorscale=colourscale))
    map_fig.update_layout (title="Temperature Map at {}th ms.".format(timeslice), width=1000, height=600) 
    map_fig.update_xaxes (title="Coordinates X", ticks="outside")
    map_fig.update_yaxes (title="Coordinates Y", ticks="outside")
    
    return map_fig
      
# ---------------- #

app.run_server (mode="external")